<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e5b52bd0dca3499273e994f617b48646b5c87d0ce666c5f436b7f1e5b3d7b5d2
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-01 14:43:21
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.28 L
Current:  1.38 C
-------------------
Today PnL: 1.62 L (1.19%)
Current PnL: -19.08 L (-12.83%)
CY Booked + Current PnL: -10.04 L (-6.75%)
-------------------
Total profit:  3.19 L
Total loss:  -22.27 L
-------------------
Total Booked + Current PnL: 16.94 L (13.92%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.54%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 80.46 L (58.24%)
Deployed:  1.22 C
Current:  1.38 C
CAGR/XIRR %: 8.04%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-16.60,62.0,H-LC,7.93,114864.0,10552.0,6099.0,1.98,10.12,5.31,15.96,9.0,1.73,0.86,28.72,X5K,ATH,METALS
53,MEDANTA,1486.00,14.09,60.0,H-MC,15.17,162910.0,36710.0,9465.0,2.21,29.09,5.81,36.59,108.0,3.88,1.22,45.30,XY24,NTT,HEALTHCARE
26,DIXON,18940.15,-30.00,66.0,H-MC,6.63,140088.0,25152.0,11431.0,4.92,21.88,8.16,31.83,81.0,2.20,1.05,45.20,X40N,ATH,IT
81,TTKPRESTIG,770.00,104.70,68.0,M-SC,7.65,88189.0,-12588.0,12682.0,2.50,-12.49,14.38,0.09,245.0,-0.99,0.66,15.86,OX40N,NTT,DURABLES
4,APOLLOHOSP,8285.00,-15.41,55.0,H-LC,8.48,168916.0,12739.0,13361.0,0.89,8.16,7.91,16.71,22.0,0.95,1.26,27.02,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,ICICIPRULI,790.00,-20.42,40.0,H-MC,2.24,136990.0,1202.0,41549.0,1.05,0.89,30.33,31.48,107.0,0.03,1.02,13.11,X40,ATH,INSURANCE
38,ICICIGI,2260.25,-20.81,35.0,H-MC,4.76,136182.0,49.0,31077.0,0.04,0.04,22.82,22.86,91.0,0.00,1.02,12.65,X40,ATH,INSURANCE
54,NATIONALUM,247.44,-49.46,54.0,H-MC,1.37,101092.0,588.0,30055.0,2.45,0.59,29.73,30.49,79.0,0.02,0.76,33.18,MH,ATH,METALS


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-41.42,41.0,H-LC,13.69,244714.0,-16905.0,136110.0,1.98,-6.46,55.62,45.57,7.0,-0.12,1.83,4.13,AR,NTT,ELECTRICAL
33,HCLTECH,1943.91,0.65,42.0,H-LC,10.06,230115.0,-11809.0,75087.0,0.75,-4.88,32.63,26.15,8.0,-0.16,1.72,8.74,X40,ATH,IT
47,JIOFIN,387.00,-0.83,42.0,H-LC,14.95,219345.0,1939.0,51546.0,0.53,0.89,23.50,24.61,48.0,0.04,1.64,56.20,XY24,BTT,FINANCE
45,ITC,452.00,-38.77,45.0,H-LC,3.11,198241.0,-1897.0,22778.0,-1.06,-0.95,11.49,10.44,4.0,-0.08,1.48,4.56,X40,NTT,FMCG
87,VBL,671.64,-12.66,45.0,H-LC,7.77,314662.0,-1180.0,113845.0,1.24,-0.37,36.18,35.67,5.0,-0.01,2.35,13.43,X40N,ATH,FMCG


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,NESTLEIND,1377.0,-10.77,56.0,H-LC,2.66,274459.0,9033.0,47756.0,1.44,3.40,17.40,21.40,11.0,0.19,2.05,10.27,XY25,NTT,FMCG
51,LTIM,7230.2,-5.78,52.0,H-LC,2.70,233325.0,-12826.0,92023.0,1.00,-5.21,39.44,32.18,16.0,-0.14,1.74,28.95,X200,ATH,IT
45,ITC,452.0,-38.77,45.0,H-LC,3.11,198241.0,-1897.0,22778.0,-1.06,-0.95,11.49,10.44,4.0,-0.08,1.48,4.56,X40,NTT,FMCG
20,CIPLA,1795.0,-18.41,62.0,H-LC,4.71,217611.0,13111.0,28311.0,-0.06,6.41,13.01,20.25,10.0,0.46,1.63,15.70,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-12.15,58.0,H-LC,7.24,221637.0,-30131.0,152353.0,1.19,-11.97,68.74,48.55,27.0,-0.20,1.66,19.22,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,VAIBHAVGBL,521.00,41.60,48.0,H-SC,11.93,94438.0,-57553.0,131675.0,6.53,-37.87,139.43,48.77,125.0,-0.44,0.71,9.92,XR,NTT,JEWELLERY
26,DIXON,18940.15,-30.00,66.0,H-MC,6.63,140088.0,25152.0,11431.0,4.92,21.88,8.16,31.83,81.0,2.20,1.05,45.20,X40N,ATH,IT
36,HINDZINC,730.22,16.37,55.0,M-LC,3.63,189725.0,-15351.0,127913.0,4.03,-7.49,67.42,54.89,52.0,-0.12,1.42,14.81,X5K,ATH,METALS
67,SFL,1287.00,40.61,61.0,M-SC,9.01,185554.0,-76664.0,146495.0,3.92,-29.24,78.95,26.63,229.0,-0.52,1.39,15.37,XY24,NTT,MISC
60,RELAXO,1176.00,-31.84,62.0,H-SC,7.80,63277.0,-52261.0,87253.0,3.51,-45.23,137.89,30.28,136.0,-0.60,0.47,24.40,X40N,NTT,FOOTWEAR


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-39.64,66.0,L-SC,29.21,90413.0,-23136.0,63190.0,-5.88,-20.38,69.89,35.27,268.0,-0.37,0.68,121.29,XR,NTT,HOTELS
6,ASIANTILES,137.0,6488.89,40.0,L-SC,5.48,72289.0,-21521.0,98140.0,-1.07,-22.94,135.76,81.67,269.0,-0.22,0.54,39.99,XR,NTT,CERAMICS
45,ITC,452.0,-38.77,45.0,H-LC,3.11,198241.0,-1897.0,22778.0,-1.06,-0.95,11.49,10.44,4.0,-0.08,1.48,4.56,X40,NTT,FMCG
50,LAOPALA,464.0,105.71,40.0,H-SC,2.79,65468.0,-35378.0,61206.0,-1.05,-35.08,93.49,25.61,142.0,-0.58,0.49,21.38,AR,NTT,CERAMICS
41,INDIGOPNTS,1408.0,105.92,38.0,M-SC,2.48,135148.0,-39411.0,39450.0,-0.89,-22.58,29.19,0.02,221.0,-1.00,1.01,17.56,OX40N,NTT,PAINTS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,104.70,68.0,M-SC,7.65,88189.0,-12588.0,12682.0,2.50,-12.49,14.38,0.09,245.0,-0.99,0.66,15.86,OX40N,NTT,DURABLES
48,KANSAINER,340.0,-68.52,52.0,H-SC,3.54,217152.0,-52515.0,88859.0,0.54,-19.47,40.92,13.47,138.0,-0.59,1.62,9.83,XY24,NTT,PAINTS
82,UJJIVANSFB,60.0,80.86,40.0,H-SC,14.32,114615.0,-27864.0,47382.0,-0.54,-19.56,41.34,13.70,163.0,-0.59,0.86,35.28,OX40N,NTT,BANKS
19,CERA,9475.0,-20.76,41.0,H-SC,3.06,113148.0,-30655.0,57400.0,-0.40,-21.32,50.73,18.60,149.0,-0.53,0.85,24.86,OX40N,NTT,CERAMICS
41,INDIGOPNTS,1408.0,105.92,38.0,M-SC,2.48,135148.0,-39411.0,39450.0,-0.89,-22.58,29.19,0.02,221.0,-1.00,1.01,17.56,OX40N,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-26.44,50.0,H-LC,15.05,292575.0,-53381.0,177242.0,0.90,-15.43,60.58,35.80,1.0,-0.30,2.19,3.65,X40,BTT,IT
43,INFY,2275.00,-17.80,50.0,H-LC,11.56,319564.0,6676.0,165023.0,2.09,2.13,51.64,54.87,3.0,0.04,2.39,8.92,X40,BTT,IT
45,ITC,452.00,-38.77,45.0,H-LC,3.11,198241.0,-1897.0,22778.0,-1.06,-0.95,11.49,10.44,4.0,-0.08,1.48,4.56,X40,NTT,FMCG
87,VBL,671.64,-12.66,45.0,H-LC,7.77,314662.0,-1180.0,113845.0,1.24,-0.37,36.18,35.67,5.0,-0.01,2.35,13.43,X40N,ATH,FMCG
1,ABB,7934.00,-41.42,41.0,H-LC,13.69,244714.0,-16905.0,136110.0,1.98,-6.46,55.62,45.57,7.0,-0.12,1.83,4.13,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,14.09,60.0,H-MC,15.17,162910.0,36710.0,9465.0,2.21,29.09,5.81,36.59,108.0,3.88,1.22,45.30,XY24,NTT,HEALTHCARE
34,HEROMOTOCO,5949.07,-1.09,75.0,H-MC,17.81,183785.0,32872.0,24425.0,3.21,21.78,13.29,37.97,93.0,1.35,1.37,52.60,AR,ATH,AUTO
26,DIXON,18940.15,-30.00,66.0,H-MC,6.63,140088.0,25152.0,11431.0,4.92,21.88,8.16,31.83,81.0,2.20,1.05,45.20,X40N,ATH,IT
27,DMART,5201.00,-5.89,76.0,H-LC,13.48,234955.0,50474.0,19901.0,0.83,27.36,8.47,38.14,38.0,2.54,1.75,40.87,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,14.09,60.0,H-MC,15.17,162910.0,36710.0,9465.0,2.21,29.09,5.81,36.59,108.0,3.88,1.22,45.30,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,12.54,65.0,H-MC,3.74,194130.0,29460.0,38515.0,3.17,17.89,19.84,41.28,89.0,0.76,1.45,27.30,X40N,ATH,AC
88,VOLTAS,1918.49,1.72,67.0,H-MC,5.87,213285.0,21543.0,74479.0,3.49,11.24,34.92,50.08,99.0,0.29,1.59,18.80,XY25,ATH,AC
34,HEROMOTOCO,5949.07,-1.09,75.0,H-MC,17.81,183785.0,32872.0,24425.0,3.21,21.78,13.29,37.97,93.0,1.35,1.37,52.60,AR,ATH,AUTO
18,CAMS,5211.76,-5.61,49.0,H-SC,5.83,107327.0,5323.0,38606.0,3.45,5.22,35.97,43.06,122.0,0.14,0.80,23.23,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-1.09,75.0,H-MC,17.81,183785.0,32872.0,24425.0,3.21,21.78,13.29,37.97,93.0,1.35,1.37,52.60,AR,ATH,AUTO
40,INDIAMART,4850.92,-50.72,51.0,H-SC,12.52,137694.0,14358.0,119408.0,0.03,11.64,86.72,108.45,119.0,0.12,1.03,36.60,AR,ATH,MISC
89,WHIRLPOOL,2270.00,-41.55,52.0,M-SC,5.48,98790.0,7292.0,71465.0,1.28,7.97,72.34,86.07,223.0,0.10,0.74,43.46,XR,NTT,DURABLES
90,WIPRO,420.00,-11.52,51.0,M-LC,7.59,155112.0,4167.0,105290.0,0.31,2.76,67.88,72.51,53.0,0.04,1.16,8.95,XR,NTT,IT
54,NATIONALUM,247.44,-49.46,54.0,H-MC,1.37,101092.0,588.0,30055.0,2.45,0.59,29.73,30.49,79.0,0.02,0.76,33.18,MH,ATH,METALS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-6.19,38.0,H-SC,16.47,87277.0,-13686.0,108215.0,1.21,-13.56,123.99,93.63,148.0,-0.13,0.65,30.00,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,66.0,L-SC,29.21,90413.0,-23136.0,63190.0,-5.88,-20.38,69.89,35.27,268.0,-0.37,0.68,121.29,XR,NTT,HOTELS
52,MASFIN,398.61,-18.25,51.0,H-SC,12.90,93000.0,-4980.0,26579.0,-0.03,-5.08,28.58,22.05,152.0,-0.19,0.69,35.18,XR,ATH,FINANCE
35,HINDALCO,756.01,-16.60,62.0,H-LC,7.93,114864.0,10552.0,6099.0,1.98,10.12,5.31,15.96,9.0,1.73,0.86,28.72,X5K,ATH,METALS
82,UJJIVANSFB,60.00,80.86,40.0,H-SC,14.32,114615.0,-27864.0,47382.0,-0.54,-19.56,41.34,13.70,163.0,-0.59,0.86,35.28,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,66.0,L-SC,29.21,90413.0,-23136.0,63190.0,-5.88,-20.38,69.89,35.27,268.0,-0.37,0.68,121.29,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,-1.09,75.0,H-MC,17.81,183785.0,32872.0,24425.0,3.21,21.78,13.29,37.97,93.0,1.35,1.37,52.60,AR,ATH,AUTO
14,BLUESTARCO,2326.38,12.54,65.0,H-MC,3.74,194130.0,29460.0,38515.0,3.17,17.89,19.84,41.28,89.0,0.76,1.45,27.30,X40N,ATH,AC
27,DMART,5201.00,-5.89,76.0,H-LC,13.48,234955.0,50474.0,19901.0,0.83,27.36,8.47,38.14,38.0,2.54,1.75,40.87,X40N,NTT,FMCG
60,RELAXO,1176.00,-31.84,62.0,H-SC,7.80,63277.0,-52261.0,87253.0,3.51,-45.23,137.89,30.28,136.0,-0.60,0.47,24.40,X40N,NTT,FOOTWEAR


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.72
1,25,43.12
2,50,72.75


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    36.92
LC    36.76
MC    26.33
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.16
X40      15.48
X40N     14.31
XY25      9.78
AR        9.30
XR        8.93
OX40N     7.27
X5K       2.28
X200      1.74
MH        1.69
SR        1.07
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.59
H-MC    23.46
H-SC    22.20
M-SC    13.16
M-LC     7.13
M-MC     2.55
L-SC     1.56
L-LC     1.04
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.12,-0.77,33.60
IT,13.37,-15.39,72.63
BANKS,7.26,-17.35,67.17
FINANCE,7.21,-24.66,70.62
PAINTS,5.94,-16.16,42.16
MISC,5.86,-17.15,84.30
ELECTRICAL,5.70,-8.01,67.54
AUTO,4.60,-9.74,57.48
AC,3.76,4.00,28.51


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2868792.0,23
AR,1233438.0,10
XR,1097849.0,13
X40,896349.0,10
X40N,517669.0,11
OX40N,484813.0,10
XY25,430242.0,7
SR,200286.0,2
X5K,134012.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2801955.0,27
M-SC,1484294.0,17
H-LC,1405231.0,17
H-MC,1235676.0,18
M-LC,425088.0,5
M-MC,336687.0,2
L-SC,252557.0,3
L-MC,59765.0,1
L-LC,44718.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,947783.0,6
M-SC,XY24,868625.0,7
H-SC,AR,669869.0,5
H-LC,X40,592483.0,5
H-SC,XR,529022.0,6
H-MC,XY24,487447.0,5
H-LC,AR,377946.0,2
M-MC,XY24,336687.0,2
H-MC,X40,222483.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
